In [32]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

from time import time
import timeit 

In [9]:
dfes = pd.read_csv('RiverRoad_Quan_RF.csv')
dfes

,GRID_ID,qu_dem_MIN,qu_dem_MAX,qu_dem_RANGE,qu_dem_MEAN,qu_dem_STD,qu_dem_SUM,qu_dsm_MIN,qu_dsm_MAX,qu_dsm_RANGE,...,qu_curpl_SUM,qu_curpr_MIN,qu_curpr_MAX,qu_curpr_RANGE,qu_curpr_MEAN,qu_curpr_STD,qu_curpr_SUM,wetland_type,FID_RiverRd_AOI,wetland
0,AA-10,0.691847,1.731059,1.039212,1.117221,0.202755,313.939159,0.731374,6.256660,5.525286,...,1688.477507,-561.943237,585.859314,1147.802551,-6.984654,173.043352,-1941.733846,nonwetland,0,nonwetland
1,AA-11,0.641888,1.487661,0.845773,1.059745,0.147762,295.668776,0.661373,6.679267,6.017894,...,-1129.404510,-841.298767,679.957519,1521.256287,0.069758,168.483844,19.392675,nonwetland,0,nonwetland
2,AA-12,0.770032,1.507396,0.737363,1.176038,0.160304,329.290651,0.770032,5.681332,4.911300,...,-994.710507,-614.981445,614.381775,1229.363220,-9.649778,201.707961,-2692.288038,nonwetland,0,nonwetland
3,AA-13,0.761148,1.413489,0.652341,1.131694,0.115760,316.874198,0.761148,1.462370,0.701221,...,-2264.176594,-577.830017,633.895386,1211.725403,-1.244892,180.105412,-347.324849,nonwetland,0,nonwetland
4,AA-14,0.696000,1.392386,0.696386,1.160470,0.116413,322.610787,0.667013,1.392386,0.725372,...,-571.203099,-574.029480,501.253113,1075.282593,-1.556956,170.969213,-435.947672,nonwetland,0,nonwetland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8606,Z-95,0.851000,1.676798,0.825798,1.409729,0.230505,390.494948,0.851000,1.676798,0.825798,...,106.915450,-692.625366,738.732422,1431.357788,4.547107,170.083954,1268.642730,PEM1Rd,1,wetland
8607,Z-96,0.723952,1.662994,0.939042,1.425776,0.219366,396.365613,0.723952,1.662994,0.939042,...,-327.422941,-390.115845,542.452454,932.568298,10.324390,134.407219,2890.829178,PEM1Rd,1,wetland
8608,Z-97,0.836438,1.653542,0.817103,1.452142,0.180319,406.599629,0.836438,1.653542,0.817103,...,279.791020,-728.279236,753.877563,1482.156799,1.897032,165.294550,527.374799,PEM1Rd,1,wetland
8609,Z-98,0.997941,1.636024,0.638083,1.461971,0.166600,406.427984,0.997941,1.636024,0.638083,...,-313.958618,-622.526367,402.200531,1024.726898,4.975549,115.808141,1383.202760,PEM1Rd,1,wetland


In [3]:
df_corr = pd.concat([dfes[dfes.columns[1:62]], dfes['FID_RiverRd_AOI']], axis=1)
corr = df_corr.corr()
df_corr = pd.DataFrame(corr['FID_RiverRd_AOI'])

df_corr = df_corr[:-1]
df_corr

,FID_RiverRd_AOI
qu_dem_MIN,-0.569517
qu_dem_MAX,-0.517710
qu_dem_RANGE,0.222622
qu_dem_MEAN,-0.533536
qu_dem_STD,0.122975
qu_dem_SUM,-0.533909
qu_dsm_MIN,-0.473204
qu_dsm_MAX,-0.471220
qu_dsm_RANGE,-0.416919
qu_dsm_MEAN,-0.443299


In [22]:
X = dfes.loc[:, ["qu_dem_MIN", "qu_dem_MAX"]]
y = dfes.loc[:, "wetland"]

In [42]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

lda = LinearDiscriminantAnalysis()
model = lda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.79048172 0.77003484 0.76945412 0.79210221 0.78164925]

Avg accuracy: 0.7807444272104607
---Run time is 0.10769759999993767 seconds ---


In [43]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

qda = QuadraticDiscriminantAnalysis()
model = qda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.81427742 0.79268293 0.79210221 0.80836237 0.79674797]

Avg accuracy: 0.8008345786965041
---Run time is 0.07057810000014797 seconds ---


In [25]:
X = dfes.loc[:, ["qu_hyddem_SUM", "qu_dsm_MIN"]]
y = dfes.loc[:, "wetland"]

In [41]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

lda = LinearDiscriminantAnalysis()
model = lda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.79048172 0.77003484 0.76945412 0.79210221 0.78164925]

Avg accuracy: 0.7807444272104607
---Run time is 0.1243082000000868 seconds ---


In [44]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

qda = QuadraticDiscriminantAnalysis()
model = qda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.81427742 0.79268293 0.79210221 0.80836237 0.79674797]

Avg accuracy: 0.8008345786965041
---Run time is 0.05823560000044381 seconds ---


In [15]:
X = dfes.loc[:, ["qu_dsm_MAX", "qu_dsm_RANGE"]]
y = dfes.loc[:, "wetland"]

In [46]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

lda = LinearDiscriminantAnalysis()
model = lda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.79048172 0.77003484 0.76945412 0.79210221 0.78164925]

Avg accuracy: 0.7807444272104607
---Run time is 0.10384019999992233 seconds ---


In [47]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

qda = QuadraticDiscriminantAnalysis()
model = qda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.81427742 0.79268293 0.79210221 0.80836237 0.79674797]

Avg accuracy: 0.8008345786965041
---Run time is 0.06754339999997683 seconds ---


In [29]:
X = dfes.loc[:, ["qu_hyddem_MIN", "qu_hyddem_MAX"]]
y = dfes.loc[:, "wetland"]

In [48]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

lda = LinearDiscriminantAnalysis()
model = lda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.79048172 0.77003484 0.76945412 0.79210221 0.78164925]

Avg accuracy: 0.7807444272104607
---Run time is 0.10843160000013086 seconds ---


In [49]:
start_time = timeit.default_timer()
crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)

qda = QuadraticDiscriminantAnalysis()
model = qda.fit(X, y)

scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

elapsed = timeit.default_timer() - start_time
print("---Run time is %s seconds ---" % elapsed)

Accuracy of each fold: 
 [0.81427742 0.79268293 0.79210221 0.80836237 0.79674797]

Avg accuracy: 0.8008345786965041
---Run time is 0.05847559999983787 seconds ---
